#### Imports

In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
from env import host, username, password

# Exercises

1. Read the case, department, and source data into their own spark dataframes.
2. Let's see how writing to the local disk works in spark:
    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [9]:
# Generate spark object to initialize a local Spark JVM process
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 13:17:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/05 13:17:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
def get_connection_url(db, user=username, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [7]:
url = get_connection_url("311_data")

---
### Case

In [10]:
query = """SELECT * FROM cases"""
case = pd.read_sql(query, url)
case = spark.createDataFrame(case)
case.show(4)

23/07/05 13:19:16 WARN TaskSetManager: Stage 0 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/05 13:19:20 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO| -2.012604167|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|2215  GOL

In [11]:
case.show(2, vertical=True)

23/07/05 13:34:00 WARN TaskSetManager: Stage 1 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/05 13:34:04 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [14]:
# case.write.json('case_json', mode='overwrite')
# case.write.csv('case_csv', mode='overwrite')

23/07/05 13:38:41 WARN TaskSetManager: Stage 5 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/05 13:38:48 WARN TaskSetManager: Stage 6 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


In [17]:
case.schema

StructType([StructField('case_id', LongType(), True), StructField('case_opened_date', StringType(), True), StructField('case_closed_date', StringType(), True), StructField('SLA_due_date', StringType(), True), StructField('case_late', StringType(), True), StructField('num_days_late', DoubleType(), True), StructField('case_closed', StringType(), True), StructField('dept_division', StringType(), True), StructField('service_request_type', StringType(), True), StructField('SLA_days', DoubleType(), True), StructField('case_status', StringType(), True), StructField('source_id', StringType(), True), StructField('request_address', StringType(), True), StructField('council_district', LongType(), True)])

In [20]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')
case.show(2, vertical=True)

23/07/05 13:58:54 WARN TaskSetManager: Stage 11 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.
23/07/05 13:58:58 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 11 (TID 53): Attempting to kill Python Worker


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [21]:
case = case.withColumnRenamed('SLA_days', 'case_days')
case.show(2, vertical=True)

23/07/05 13:59:01 WARN TaskSetManager: Stage 12 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 case_days            | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

23/07/05 13:59:05 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 12 (TID 54): Attempting to kill Python Worker


In [22]:
# demonstrating we only have yes/no in each field
case.groupby('case_closed', 'case_late').count().show()

23/07/05 13:59:15 WARN TaskSetManager: Stage 13 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [23]:
#Let's create two new boolean columns
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
        "case_late", expr('case_late == "YES"'))

#Select just the two columns
case.select("case_closed", "case_late").show()

23/07/05 13:59:31 WARN TaskSetManager: Stage 16 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
+-----------+---------+
only showing top 20 rows



23/07/05 13:59:35 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 16 (TID 64): Attempting to kill Python Worker


In [25]:
case.groupBy('council_district').count().show()

23/07/05 14:01:11 WARN TaskSetManager: Stage 18 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+------+
|council_district| count|
+----------------+------+
|               0|  3830|
|               7| 72445|
|               6| 74095|
|               9| 40916|
|               5|114609|
|               1|119309|
|              10| 62926|
|               3|102706|
|               8| 42345|
|               2|114745|
|               4| 93778|
+----------------+------+



In [26]:
case = case.withColumn('council_district', col('council_district').cast('string'))

In [27]:
#Before
case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

23/07/05 14:01:23 WARN TaskSetManager: Stage 21 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows



23/07/05 14:01:28 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 21 (TID 82): Attempting to kill Python Worker


In [28]:
#After

fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

23/07/05 14:01:34 WARN TaskSetManager: Stage 22 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



23/07/05 14:01:38 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 22 (TID 83): Attempting to kill Python Worker


In [29]:
#After
case = case.withColumn('request_address', trim(lower(case.request_address)))

case.select("request_address").show(5, 50)

23/07/05 14:01:47 WARN TaskSetManager: Stage 23 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------------------+
|                      request_address|
+-------------------------------------+
| 2315  el paso st, san antonio, 78207|
|  2215  goliad rd, san antonio, 78223|
|102  palfrey st w, san antonio, 78223|
| 114  la garde st, san antonio, 78223|
|734  clearview dr, san antonio, 78228|
+-------------------------------------+
only showing top 5 rows



23/07/05 14:01:51 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 23 (TID 84): Attempting to kill Python Worker


In [30]:
case = case.withColumn('num_weeks_late', expr('num_days_late / 7'))

case.select("num_days_late", "num_weeks_late").show(5)

23/07/05 14:01:55 WARN TaskSetManager: Stage 24 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------------+
|num_days_late|      num_weeks_late|
+-------------+--------------------+
| -998.5087616|        -142.6441088|
| -2.012604167|        -0.287514881|
| -3.022337963|-0.43176256614285713|
| -15.01148148| -2.1444973542857144|
|  0.372164352|0.053166335999999995|
+-------------+--------------------+
only showing top 5 rows



23/07/05 14:01:59 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 24 (TID 85): Attempting to kill Python Worker


In [31]:
case = case.withColumn('council_district', col('council_district').cast('int'))

# '%03d' means at least 3 digits, pad with 0s
#
# In order to use the format_string function the way we are, we'll need to
# convert council_district back to an integer temporarily, but the final output
# will be a string.
case = case.withColumn('council_district',
                  format_string('%03d', col('council_district').cast('int')))

case.select("council_district").show(5)

23/07/05 14:02:03 WARN TaskSetManager: Stage 25 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


+----------------+
|council_district|
+----------------+
|             005|
|             003|
|             003|
|             003|
|             007|
+----------------+
only showing top 5 rows



23/07/05 14:02:07 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 25 (TID 86): Attempting to kill Python Worker


In [32]:
case = (
# calculates the number of days between the current timestamp & case_opened_date
    case.withColumn(
        'case_age', datediff(current_timestamp(), 'case_opened_date')
    )
# Calculates the number of days between the case_closed_date & case_opened_date
    .withColumn(
        'days_to_closed', datediff('case_closed_date', 'case_opened_date')
    )
# since case_closed is boolean, we can check if it's T/F;
#If the case is not closed, it takes the value from the "case_age" column
#Otherwise, it takes the value from the "days_to_closed" column    
    .withColumn(
        'case_lifetime', 
            when(expr('! case_closed'), col('case_age')).otherwise(col('days_to_closed'))

    )
)

In [34]:
case.show(2, vertical=True)

23/07/05 14:02:52 WARN TaskSetManager: Stage 26 contains a task of very large size (18868 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 case_days            | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 case_age             | 2011                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

23/07/05 14:02:56 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 26 (TID 87): Attempting to kill Python Worker


---
### Dept

In [12]:
query = """SELECT * FROM dept"""
dept = pd.read_sql(query, url)
dept = spark.createDataFrame(dept)
dept.show(4)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 4 rows



In [15]:
# dept.write.json('dept_json', mode='overwrite')
# dept.write.csv('dept_csv', mode='overwrite')

In [18]:
dept.schema

StructType([StructField('dept_division', StringType(), True), StructField('dept_name', StringType(), True), StructField('standardized_dept_name', StringType(), True), StructField('dept_subject_to_SLA', StringType(), True)])

---
### Source

In [13]:
query = """SELECT * FROM source"""
source = pd.read_sql(query, url)
source = spark.createDataFrame(source)
source.show(4)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
+-----+---------+----------------+
only showing top 4 rows



In [16]:
# source.write.json('source_json', mode='overwrite')
# source.write.csv('source_csv', mode='overwrite')

In [19]:
source.schema

StructType([StructField('index', LongType(), True), StructField('source_id', StringType(), True), StructField('source_username', StringType(), True)])